In [3]:
from dotenv import load_dotenv
from pprint import pprint

load_dotenv("key.env",verbose=True)

True

# Chap.4 LangChainの基礎

## Language Models

### Completions API (Deprecated)

#### LLMs

In [2]:
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)


result = llm("自己紹介してください")
print(result)

/Users/tylerkurahashi/Documents/GitHub/langchain_study/venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(




私は、山田太郎と申します。東京都出身で、現在は大学生として都内の大学に通っています。趣味はスポーツ観戦や音楽鑑賞で、特にサッカーやロックバンドが好きです。将来の夢は、国際的な企業で働くことで、留学経験も積んで自分を磨きたいと思っています。また、人とのコミュニケーションを大切にし、常に新しいことに挑戦することで成長していきたいと考えています。よろしくお願いします。


### Chat Completions API

#### Chat models

- AIMessage
- HumanMessage
- SystemMessage

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content="私の名前がわかりますか？")
]
result = chat(messages)
print(result.content)


はい、おっしゃった通り、あなたの名前はジョンさんですね。他に何かお手伝いできることがありますか？


#### Streaming with Callbacks

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

messages = [HumanMessage(content="自己紹介してください")]
result = chat(messages) # type:ignore


はじめまして、私はAIアシスタントです。自然言語処理技術を用いて、様々な質問や会話に対応することができます。お手伝いが必要なことがあれば、遠慮なくお知らせください。どうぞよろしくお願いいたします。

## Prompts

### PromptTemplate

In [6]:
from langchain.prompts import PromptTemplate

template = """
以下の料理のレシピを考えてください

料理名: {dish}
"""

prompt = PromptTemplate(
    input_variables=["dish"],
    template=template
)

result = prompt.format(dish="カレー")
print(result)


以下の料理のレシピを考えてください

料理名: カレー



### ChatPromptTemplate

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.schema import HumanMessage, SystemMessage

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("あなたは{country}料理のプロフェッショナルです。"),
        HumanMessagePromptTemplate.from_template("以下の料理のレシピを考えてください。\n\n料理名: {dish}"),
    ]
)

messages = chat_prompt.format_prompt(country="イギリス", dish="肉じゃが").to_messages()
pprint(messages)

/Users/tylerkurahashi/Documents/GitHub/langchain_study/venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[SystemMessage(content='あなたはイギリス料理のプロフェッショナルです。', additional_kwargs={}),
 HumanMessage(content='以下の料理のレシピを考えてください。\n\n料理名: 肉じゃが', additional_kwargs={}, example=False)]


## Output parsers
LLMの出力からPythonオブジェクトを作成する

### PydanticOutputParser

In [3]:
from pydantic import BaseModel, Field
from typing import List

class Recipe(BaseModel):
    ingredients: List[str] = Field(
        description="ingredients of the dish",
    )
    steps: List[str] = Field(
        description="steps to make the dish",
    )

In [4]:
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Recipe)

In [5]:
format_instructions = parser.get_format_instructions()
pprint(format_instructions)

('The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"ingredients": {"description": "ingredients of the dish", '
 '"items": {"type": "string"}, "title": "Ingredients", "type": "array"}, '
 '"steps": {"description": "steps to make the dish", "items": {"type": '
 '"string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", '
 '"steps"]}\n'
 '```')


In [6]:
template = """
料理のレシピを考えてください

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": format_instructions}
)

In [7]:
formatted_prompt = prompt.format(dish="カレー")
pprint(formatted_prompt)

('\n'
 '料理のレシピを考えてください\n'
 '\n'
 'The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"ingredients": {"description": "ingredients of the dish", '
 '"items": {"type": "string"}, "title": "Ingredients", "type": "array"}, '
 '"steps": {"description": "steps to make the dish", "items": {"type": '
 '"string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", '
 '"steps"]}\n'
 '```\n'
 '\n'
 '料理名: カレー\n')


In [8]:
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

messages = [HumanMessage(content=formatted_prompt)]
output = chat(messages)
pprint(output.content)

('```json\n'
 '{\n'
 '    "ingredients": [\n'
 '        "玉ねぎ",\n'
 '        "人参",\n'
 '        "じゃがいも",\n'
 '        "牛肉",\n'
 '        "カレールー",\n'
 '        "水"\n'
 '    ],\n'
 '    "steps": [\n'
 '        "1. 材料を準備する。",\n'
 '        "2. 玉ねぎ、人参、じゃがいもを切る。",\n'
 '        "3. 牛肉を炒める。",\n'
 '        "4. 野菜を加えて炒める。",\n'
 '        "5. 水を加えて煮込む。",\n'
 '        "6. カレールーを溶かして加える。",\n'
 '        "7. 全体を混ぜて完成。"\n'
 '    ]\n'
 '}\n'
 '```')


In [9]:
recipe = parser.parse(output.content)
print(type(recipe))
pprint(recipe)

<class '__main__.Recipe'>
Recipe(ingredients=['玉ねぎ', '人参', 'じゃがいも', '牛肉', 'カレールー', '水'], steps=['1. 材料を準備する。', '2. 玉ねぎ、人参、じゃがいもを切る。', '3. 牛肉を炒める。', '4. 野菜を加えて炒める。', '5. 水を加えて煮込む。', '6. カレールーを溶かして加える。', '7. 全体を混ぜて完成。'])


## Chains

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: List[str] = Field(description="ingredients of the dish")
    steps: List[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

template = """
料理のレシピを考えてください

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [11]:
from langchain import LLMChain

chain = LLMChain(prompt=prompt, llm=chat, output_parser=output_parser)

recipe = chain.run(dish="カレー")
print(type(recipe))
pprint(recipe)

<class '__main__.Recipe'>
Recipe(ingredients=['玉ねぎ', '人参', 'じゃがいも', '牛肉', 'カレールー', '水'], steps=['1. 材料を準備する。', '2. 玉ねぎ、人参、じゃがいもを切る。', '3. 牛肉を炒める。', '4. 野菜を加えて炒める。', '5. 水を加えて煮込む。', '6. カレールーを溶かして加える。', '7. 全体を混ぜて完成。'])


In [12]:
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

cot_template = """
以下の質問に回答してください

質問: {question}

ステップバイステップで考えましょう。
"""

cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template,
)

cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

In [13]:
summarize_template = """
以下の文章を結論だけ一言に要約してください

{input}
"""

summarize_prompt = PromptTemplate(
    input_variables=["input"],
    template=summarize_template
)

summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

In [14]:
from langchain.chains import SimpleSequentialChain

cot_summarize_chain = SimpleSequentialChain(
    chains=[cot_chain, summarize_chain],
)

result = cot_summarize_chain(
    "私は市場に行って10個のりんごを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのりんごを買って1つ食べました。残りは何個ですか？"
)

print(result["output"])

残りのりんごは10個。


## Memory

### ConversationBufferMemory

In [15]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True: # -> infinite loop
    user_message = input("You: ")
    ai_message = conversation.run(input=user_message)
    print(f"AI: {ai_message}")

AI: こんにちは、Tylerさん。私はAIです。どのようにお手伝いできますか？
AI: はい、Tylerさんの名前を知っています。それはあなたの自己紹介で教えてくれたからです。他に何か知りたいことがありますか？
AI: 何か質問があれば遠慮なくどうぞ。私はできる限りお手伝いします。
AI:  了解しました。もし何か他の質問やお手伝いが必要な場合はいつでもお知らせくださいね。良い一日を！
AI:  それでは、またお会いしましょう。良い一日を！
AI:  さようなら！またお会いしましょう。
AI:  さようなら！またお会いしましょう。


# Chap.5 LangChainの活用

In [1]:
from dotenv import load_dotenv
from pprint import pprint

load_dotenv("key.env",verbose=True)

True

## Data Connection

### RAG (Retrieval Augmented Generation)

In [2]:
from langchain.document_loaders import GitLoader

def file_filter(file_path):
  return file_path.endswith(".mdx")

loader = GitLoader(
  clone_url="https://github.com/langchain-ai/langchain",
  repo_path="./langchain",
  branch="master",
  file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

/Users/tylerkurahashi/Documents/GitHub/langchain_study/venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


267


### Document transformers
- Html2TextTransformer
- EmbeddingsRedundantFilter
- OpenAIMetadataTagger
- DoctranTextTranslator
- DoctranQATransformer

In [3]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
len(docs)

Created a chunk of size 6803, which is longer than the specified 1000
Created a chunk of size 3302, which is longer than the specified 1000
Created a chunk of size 1851, which is longer than the specified 1000
Created a chunk of size 1639, which is longer than the specified 1000
Created a chunk of size 9269, which is longer than the specified 1000
Created a chunk of size 2579, which is longer than the specified 1000
Created a chunk of size 42230, which is longer than the specified 1000
Created a chunk of size 1386, which is longer than the specified 1000
Created a chunk of size 2103, which is longer than the specified 1000
Created a chunk of size 4067, which is longer than the specified 1000
Created a chunk of size 1220, which is longer than the specified 1000
Created a chunk of size 18110, which is longer than the specified 1000
Created a chunk of size 1001, which is longer than the specified 1000
Created a chunk of size 6750, which is longer than the specified 1000
Created a chunk of

856

### Text embedding models

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [5]:
query = "AWSのS3からデータを読み込むためのDocumentLoaderはありますか?"

vector = embeddings.embed_query(query)
print(len(vector))
pprint(vector)

1536
[-0.015691175959958982,
 -0.0021942492904605923,
 0.01592232835890588,
 -0.006880186455675645,
 -0.015337649638108491,
 0.03815376362484057,
 -0.015351246947613905,
 -0.02020544918814396,
 0.0012976834470359788,
 0.004068964135850377,
 0.011108918046892067,
 0.016792550579316966,
 0.017839535960653297,
 -0.009851176230866959,
 -0.007424075110601931,
 -0.01763557818071722,
 0.017118884144801765,
 -0.0016911525640505485,
 0.0034519908736550596,
 -0.04179781276996933,
 0.00889257336131582,
 -0.004531269399405463,
 -0.004160745509350638,
 -0.006982165811304965,
 -0.01709168952579094,
 0.008341886051636827,
 0.014684983438461462,
 -0.03698440245795551,
 0.019566381229324914,
 -0.006441676949416318,
 0.05392652344183203,
 -0.018682561699408413,
 0.0038140164452690017,
 -0.020898906384984664,
 -0.0021721538953449367,
 -0.026174623357537417,
 0.0009722002954718736,
 -0.007451269263951474,
 -0.013468033550952595,
 -0.016058301453960023,
 0.012387055827175298,
 -0.012414249514863557,
 0.007

### Vector stores

In [5]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)

### Retrievers

In [6]:
retriever = db.as_retriever()

In [7]:
query = "AWSのS3からデータを読み込むためのDocumentLoaderはありますか?"

context_docs = retriever.get_relevant_documents(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
pprint(first_doc.page_content)

len = 4
metadata = {'file_name': 'aws.mdx', 'file_path': 'docs/docs/integrations/platforms/aws.mdx', 'file_type': '.mdx', 'source': 'docs/docs/integrations/platforms/aws.mdx'}
('>[Amazon Simple Storage Service (Amazon '
 'S3)](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)\n'
 '> is an object storage service.\n'
 '>[AWS S3 '
 'Directory](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)\n'
 '>[AWS S3 '
 'Buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html)\n'
 '\n'
 'See a [usage example for '
 'S3DirectoryLoader](/docs/integrations/document_loaders/aws_s3_directory).\n'
 '\n'
 'See a [usage example for '
 'S3FileLoader](/docs/integrations/document_loaders/aws_s3_file).\n'
 '\n'
 '```python\n'
 'from langchain_community.document_loaders import S3DirectoryLoader, '
 'S3FileLoader\n'
 '```\n'
 '\n'
 '### Amazon Textract\n'
 '\n'
 '>[Amazon '
 'Textract](https://docs.aws.amazon.com/managedservices/latest/usergu

queryを基に文書の検索に使えそう

### RetrievalQA (Chain)

In [8]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=chat, chain_type="stuff", retriever=retriever
)

query = "AWSのS3からデータを読み込むためのDocumentLoaderはありますか?"

result = qa_chain.run(query)
print(result)

はい、AWSのS3からデータを読み込むためのDocumentLoaderとして、`S3DirectoryLoader`と`S3FileLoader`が提供されています。これらのDocumentLoaderを使用することで、AWSのS3からデータを効率的に読み込むことができます。


RetrievalQAで使用できるchain_type
- stuff <br/>
全文書を一つのプロンプトに突っ込んでLLMのinputとする
- map_reduce <br/>
それぞれの文書をそれぞれLLMにinputに入れて回答を得る、その後その回答をすべて一つのPromptに突っ込んで再度LLMのinputとする
- map_rerank <br/>
それぞれの文書をそれぞれLLMにinputに入れて回答を得る、その後その回答のうち最もスコアの高い回答を最終的な出力として採用する
- refine <br/>
文書を一つずつLLMのinputとして途中回答を得ていく手法。徐々に回答を改善していくようなケースにて採用する

## Agents

### Agentsの使用例

In [10]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"])
agent_chain = initialize_agent(
    tools, chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

result = agent_chain.run(".vscodeディレクトリにあるファイルの一覧を教えて")
print(result)

/Users/tylerkurahashi/Documents/GitHub/langchain_study/venv/lib/python3.8/site-packages/langchain/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


settings.json contains the following configurations:
{
    "python.analysis.typeCheckingMode": "basic",
    "python.analysis.autoImportCompletions": true
}


### Tools

使用可能なツールの例
- terminal
- Python_REPL
- Wikipedia
- human

In [11]:
from langchain.tools import Tool

def my_super_func(param):
    return "42"

tools = [
    Tool.from_function(
        func=my_super_func,
        name="The_Answer",
        description="Returns the answer to life, the universe, and everything",
    )
]

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain import LLMChain

summarize_template = """
以下の文章を結論だけ一言に要約してください

{input}
"""

summarize_prompt = PromptTemplate(
    input_variables=["input"],
    template=summarize_template,
)

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

tools = [
    Tool.from_function(
        func=summarize_chain.run, name="Summarizer", description="Text summarizer"
    )
]

### Toolkits

- AzureCognitiveServicesToolkit
- GmailToolkit
- JiraToolkit
- O365Toolkit
- OpenAPIToolkit
- SQLDatabaseToolkit
- VectorStoreToolkit
- ZapierToolkit

###  Function_callingを使うOpenAI Functions Agent

In [14]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"])
agent_chain = initialize_agent(tools, chat, agent=AgentType.OPENAI_FUNCTIONS) # agent typeでfunction_callingを指定できる

result = agent_chain.run(".vscodeディレクトリにあるファイルの一覧を教えて")
print(result)

/Users/tylerkurahashi/Documents/GitHub/langchain_study/venv/lib/python3.8/site-packages/langchain/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


.vscodeディレクトリには以下のファイルが含まれています:
1. settings.json


### 一度に複数ツールを使うOpenAI Multi Functions Agent

In [4]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = load_tools(["ddg-search"]) ## ImportErrorで動かせず
agent_chain = initialize_agent(tools, chat, agent=AgentType.OPENAI_MULTI_FUNCTIONS)

result = agent_chain.run("東京と大阪の天気を教えて")
print(result)

東京の天気は今日は20℃で曇りで、明日は17℃で雨の可能性があります。一方、大阪の天気は今日は晴れから曇りに変わり、明日は曇りから小雨に変わります。気温や降水確率など詳細な情報はウェザーニュースなどのサイトで確認できます。
